In [1]:
!date

Wed Apr 27 10:52:48 EDT 2022


In [2]:
pwd

'/mmfs1/data/aglinska/BC-fMRI-AE/Notebooks'

In [3]:
%%time
import os
import numpy as np
import pandas as pd
import ants

from tqdm import tqdm

CPU times: user 2.9 s, sys: 3.3 s, total: 6.2 s
Wall time: 23.6 s


In [4]:
## MAKE SFARI DF

In [5]:
big = pd.read_csv('../Data/sfari-bigout.csv')
leg = pd.read_csv('../Data/sfari0.csv')

big.sort_values(by='Unnamed: 0',inplace=True)
leg.sort_values(by='sfari_ids',inplace=True)

print(big.shape)
print(leg.shape)

(341, 53)
(208, 4)


In [6]:
slice_ = np.array([val in leg['sfari_ids'].values for val in big['Unnamed: 0'].values])
big = big.iloc[slice_]

In [7]:
assert all([big['Unnamed: 0'].values[i]==leg['sfari_ids'].values[i] for i in range(big.shape[0])]),'bad'

In [8]:
df = pd.DataFrame()
df['SFARI_ID'] = leg['sfari_ids']
df['BIDS_ID'] = leg['subID']
df['site'] = leg['site_ids']

In [9]:
cols_use = ['age_years','latest_clinical_asd_dx','svip_diagnosis_m1', 'family_type','genetic_status_16p','ados_module','age_months','best_full_scale_iq','best_nonverbal_iq','best_verbal_iq','clinical_asd_dx','ados_css','rrb_css', 'sa_css','sex','genetic_status_1q','collection', 'date_of_evaluation','eval_age_months','family', 'genetic_status_info_source','father','htwhc.bmi', 'htwhc.bmi_z_score', 'htwhc.head_circum_1','htwhc.head_circum_2','htwhc.head_circum_3', 'htwhc.head_circum_final','htwhc.head_circum_z_score','htwhc.height', 'htwhc.height_z_score','htwhc.measure.eval_age_months','htwhc.measure.measure_type','htwhc.role','htwhc.weight', 'htwhc.weight_z_score','included_in_longitudinal_dataset','individual.family','inheritance_information', 'inherited_status_info_source','initial_clinical_asd_dx', 'measure_type','mother', 'relationship_to_iip','svip_neuro_exam.background.head_circum','svip_neuro_exam.background.head_circum_perc','svip_neuro_exam.background.head_circum_z_score','svip_neuro_exam.measure.eval_age_months','svip_neuro_exam.measure.measure_type']

In [10]:
for c in range(len(cols_use)):
    df[cols_use[c]] = big[cols_use[c]].values

In [11]:
df.head()

,SFARI_ID,BIDS_ID,site,age_years,latest_clinical_asd_dx,svip_diagnosis_m1,family_type,genetic_status_16p,ados_module,age_months,...,inherited_status_info_source,initial_clinical_asd_dx,measure_type,mother,relationship_to_iip,svip_neuro_exam.background.head_circum,svip_neuro_exam.background.head_circum_perc,svip_neuro_exam.background.head_circum_z_score,svip_neuro_exam.measure.eval_age_months,svip_neuro_exam.measure.measure_type
0,14702.x3,sub-001,60,8,False,non-spectrum-dx,16p-deletion,deletion,ados-2,95,...,confirmed,False,diagnosis-summary,14702-x1,Initially identified proband,55.9,NaN,2.18,119.0,svip-neuro-exam
1,14703.x3,sub-002,50,11,True,pdd-nos-atypical-autism,16p-deletion,deletion,ados-3,133,...,confirmed,True,diagnosis-summary,14703-x1,Initially identified proband,57.2,NaN,3.29,133.0,svip-neuro-exam
2,14705.x14,sub-003,50,34,False,no-diagnosis,16p-duplication,duplication,ados-4,405,...,confirmed,False,diagnosis-summary,14705-x1,Mother,NaN,NaN,NaN,405.0,svip-neuro-exam
3,14708.x10,sub-004,60,9,False,non-spectrum-dx,16p-deletion,deletion,ados-3,113,...,confirmed,False,diagnosis-summary,14708-x7,Initially identified proband,54.8,NaN,2.08,113.0,svip-neuro-exam
4,14713.x7,sub-005,50,7,False,non-spectrum-dx,16p-deletion,deletion,ados-3,87,...,confirmed,False,diagnosis-summary,14713-x3,Initially identified proband,55.2,NaN,2.28,87.0,svip-neuro-exam


In [12]:
df.to_csv('../Data/sfari_df.csv')

In [ ]:
## Get Scan Durations

In [13]:
subs = [sub for sub in os.listdir(os.path.expanduser('~/data/sfari_bids/preprocessed/')) if sub.startswith('sub-')]
subs.sort()
subs[0:5]

['sub-001', 'sub-002', 'sub-003', 'sub-005', 'sub-006']

In [14]:
def get_duration(fn):
    try:
        im = ants.image_read(fn)
        T = im.spacing[-1]*im.shape[-1]/60
    except:
        T = 0
    return T

In [15]:
bold_temp = '~/data/sfari_bids/preprocessed/{sub}/frc-{sub}_run-1_bold.nii.gz'
bold_temp = os.path.expanduser(bold_temp)
Ts1 = [get_duration(bold_temp.format(sub=sub)) for sub in tqdm(df['BIDS_ID'].values)]

100%|██████████| 208/208 [01:40<00:00,  2.07it/s]


In [16]:
bold_temp = '~/data/sfari_bids/preprocessed/{sub}/frc-{sub}_run-2_bold.nii.gz'
bold_temp = os.path.expanduser(bold_temp)
Ts2 = [get_duration(bold_temp.format(sub=sub)) for sub in tqdm(df['BIDS_ID'].values)]

100%|██████████| 208/208 [01:28<00:00,  2.36it/s]


In [18]:
df['dur_run1'] = Ts1
df['dur_run2'] = Ts2

In [28]:
df = df.iloc[df['dur_run1'].values>4]
df

,SFARI_ID,BIDS_ID,site,age_years,latest_clinical_asd_dx,svip_diagnosis_m1,family_type,genetic_status_16p,ados_module,age_months,...,measure_type,mother,relationship_to_iip,svip_neuro_exam.background.head_circum,svip_neuro_exam.background.head_circum_perc,svip_neuro_exam.background.head_circum_z_score,svip_neuro_exam.measure.eval_age_months,svip_neuro_exam.measure.measure_type,dur_run1,dur_run2
0,14702.x3,sub-001,60,8,False,non-spectrum-dx,16p-deletion,deletion,ados-2,95,...,diagnosis-summary,14702-x1,Initially identified proband,55.9,NaN,2.18,119.0,svip-neuro-exam,5.00,4.25
2,14705.x14,sub-003,50,34,False,no-diagnosis,16p-duplication,duplication,ados-4,405,...,diagnosis-summary,14705-x1,Mother,NaN,NaN,NaN,405.0,svip-neuro-exam,4.45,3.90
5,14714.x18,sub-006,60,21,False,no-diagnosis,16p-deletion,deletion,ados-4,249,...,diagnosis-summary,14714-x12,Initially identified proband,55.8,NaN,0.49,249.0,svip-neuro-exam,6.00,6.00
8,14723.x17,sub-009,60,5,False,non-spectrum-dx,16p-duplication,duplication,ados-2,60,...,diagnosis-summary,14723-x9,Initially identified proband,NaN,NaN,NaN,60.0,svip-neuro-exam,5.20,5.30
10,14725.x46,sub-011,50,10,False,no-diagnosis,16p-duplication,duplication,ados-3,120,...,diagnosis-summary,14725-x55,Cousin,49.2,NaN,-2.35,120.0,svip-neuro-exam,4.30,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,15057.x1,sub-202,50,62,False,NaN,non-familial-control,NaN,NaN,747,...,NaN,NaN,NaN,58.0,NaN,2.02,747.0,svip-neuro-exam,4.45,4.70
202,15060.x1,sub-203,50,28,False,NaN,non-familial-control,NaN,NaN,338,...,NaN,NaN,NaN,60.0,NaN,3.42,338.0,svip-neuro-exam,5.10,3.85
204,15071.x1,sub-205,60,28,False,NaN,non-familial-control,NaN,NaN,334,...,NaN,NaN,NaN,57.5,95.0,1.67,335.0,svip-neuro-exam,6.00,5.70
206,15083.x5,sub-207,50,12,False,non-spectrum-dx,16p-deletion,deletion,ados-3,149,...,diagnosis-summary,15083-x3,Initially identified proband,58.0,NaN,2.77,149.0,svip-neuro-exam,5.05,5.35


In [29]:
df.to_csv('../Data/sfari_df_S134.csv')

In [39]:
((df['family_type'].values=='16p-deletion') * (df['genetic_status_16p'].values=='deletion')).sum()

22

In [40]:
((df['family_type'].values=='16p-duplication') * (df['genetic_status_16p'].values=='duplication')).sum()

23

In [56]:
fn_temp = '~/BC-fMRI-AE/Assets/SFARI-cmats-frc/{sub}-cmat-run-1-frc.npy'
fn_temp = os.path.expanduser(fn_temp)

In [53]:
cmats = np.array([np.load(fn_temp.format(sub=sub)) for sub in tqdm(df['BIDS_ID'].values)])

100%|██████████| 134/134 [00:00<00:00, 3560.15it/s]


In [60]:
np.save(file='../Data/cmats_SFARI_S134.npy', arr=cmats)